In [1]:
import pandas as pd
from sqlalchemy import create_engine

In [2]:
employ_file= "DP_LIVE_Employdata.csv"
GDP_file="DP_LIVE_GDPdata.csv"

In [3]:
employ_df = pd.read_csv(employ_file)
employ_df = employ_df.loc[employ_df['MEASURE'] == 'PC_WKGPOP'] #removing the measure = THND_PER; keeping only 0-100 perc. rate
employ_df

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
163,AUS,EMP,MEN,PC_WKGPOP,A,1979,82.10677,NaN
164,AUS,EMP,MEN,PC_WKGPOP,A,1980,82.25813,NaN
165,AUS,EMP,MEN,PC_WKGPOP,A,1981,82.32400,NaN
166,AUS,EMP,MEN,PC_WKGPOP,A,1982,80.50681,NaN
167,AUS,EMP,MEN,PC_WKGPOP,A,1983,76.92610,NaN
168,AUS,EMP,MEN,PC_WKGPOP,A,1984,77.40482,NaN
169,AUS,EMP,MEN,PC_WKGPOP,A,1985,77.65910,NaN
170,AUS,EMP,MEN,PC_WKGPOP,A,1986,78.21666,NaN
171,AUS,EMP,MEN,PC_WKGPOP,A,1987,77.72889,NaN
172,AUS,EMP,MEN,PC_WKGPOP,A,1988,78.56184,NaN


In [4]:
GDP_df=pd.read_csv(GDP_file)
GDP_df.head()

,LOCATION,INDICATOR,SUBJECT,MEASURE,FREQUENCY,TIME,Value,Flag Codes
0,AUS,GDP,TOT,MLN_USD,A,1960,25036.1668,NaN
1,AUS,GDP,TOT,MLN_USD,A,1961,25326.3822,NaN
2,AUS,GDP,TOT,MLN_USD,A,1962,27913.2057,NaN
3,AUS,GDP,TOT,MLN_USD,A,1963,30385.7232,NaN
4,AUS,GDP,TOT,MLN_USD,A,1964,32695.8783,NaN


In [5]:
GDP_new_df=GDP_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
GDP_new_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
0,AUS,GDP,TOT,1960,25036.1668
1,AUS,GDP,TOT,1961,25326.3822
2,AUS,GDP,TOT,1962,27913.2057
3,AUS,GDP,TOT,1963,30385.7232
4,AUS,GDP,TOT,1964,32695.8783


In [6]:
GDP_new_df=GDP_new_df.rename(columns={"Value":"GDP Value"})

In [7]:
employ_new_df=employ_df[["LOCATION","INDICATOR","SUBJECT","TIME","Value"]]
employ_new_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
163,AUS,EMP,MEN,1979,82.10677
164,AUS,EMP,MEN,1980,82.25813
165,AUS,EMP,MEN,1981,82.32400
166,AUS,EMP,MEN,1982,80.50681
167,AUS,EMP,MEN,1983,76.92610


#### Cleaning

In [8]:
# Dropping the values where subject != Total (Men; Women)
# GDP_new_df = GDP_new_df.loc[GDP_new_df['SUBJECT'] == "TOT"]

employ_new_df = employ_new_df.loc[employ_new_df['SUBJECT'] == "TOT"]
employ_new_df.head()

,LOCATION,INDICATOR,SUBJECT,TIME,Value
203,AUS,EMP,TOT,1979,64.38335
204,AUS,EMP,TOT,1980,65.17445
205,AUS,EMP,TOT,1981,65.36742
206,AUS,EMP,TOT,1982,64.16441
207,AUS,EMP,TOT,1983,61.94022


In [9]:
#Employement has 2016-Q2 & 2016-12 data - need to get grouped, averaged out and replaced on yearly basis

employ_new_df['TIME'] = pd.to_datetime(employ_new_df['TIME']).apply(lambda x : pd.Period(x, freq='M').year)

#employ_new_df.groupby(['LOCATION'], ['TIME']).mean()
#employ_new_df.groupby(['LOCATION','TIME']).reset_index()
#employ_new_df = employ_new_df.groupby(('LOCATION', 'TIME'], as_index=False).agg({"Value": "sum"})

# employ_new_df= employ_new_df.groupby(['LOCATION','TIME'], as_index=False).agg({'Value':'mean'})
employ_new_df.head()


,LOCATION,INDICATOR,SUBJECT,TIME,Value
203,AUS,EMP,TOT,1979,64.38335
204,AUS,EMP,TOT,1980,65.17445
205,AUS,EMP,TOT,1981,65.36742
206,AUS,EMP,TOT,1982,64.16441
207,AUS,EMP,TOT,1983,61.94022


In [10]:
employ_new_df = employ_new_df.loc[employ_new_df['TIME'] >= 2000]
GDP_new_df = GDP_new_df.loc[GDP_new_df['TIME'] >= 2000]



In [11]:
combined = pd.merge(GDP_new_df, employ_new_df, 
                   on = ['TIME', 'LOCATION'])
combined

,LOCATION,INDICATOR_x,SUBJECT_x,TIME,GDP Value,INDICATOR_y,SUBJECT_y,Value
0,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.09614
1,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,68.51524
2,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.11317
3,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.65611
4,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.05413
5,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,68.29255
6,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,68.52596
7,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,68.72720
8,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.04128
9,AUS,GDP,TOT,2000,5.377613e+05,EMP,TOT,69.01699
